In [79]:
import os, dotenv
if '__file__' in globals():
    parent_directory = os.path.dirname(os.path.realpath(__file__))
else:
    parent_directory = os.getcwd()

plugin_path = os.path.join(parent_directory, "plugins/")

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions
from semantic_kernel.utils.settings import openai_settings_from_dot_env
from semantic_kernel.core_plugins import MathPlugin, TimePlugin
from llm_prompts import SEQUENTIAL_PLANNER_PROMPT
from core.database_connection import Neo4jPlugin
dotenv.load_dotenv(dotenv_path='../.env') 

async def main():
    kernel = Kernel()

    service_id = "planner"
    api_key = os.environ.get("OPENAI_API_KEY")
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id="gpt-3.5-turbo-1106",
            api_key=api_key,
        ),
    )

    kernel.add_plugin(parent_directory=plugin_path, plugin_name="ChatScripts")
    kernel.add_plugin(parent_directory=plugin_path, plugin_name="CypherProcessing")
    
    kernel.add_plugins({"Neo4jCalling": Neo4jPlugin()})
    
    #-Sequential Planner-------------------------------------------
    from semantic_kernel.planners import SequentialPlanner
    planner = SequentialPlanner(kernel, service_id, prompt=SEQUENTIAL_PLANNER_PROMPT)

    #-Questions-------------------------------------------
    questions = [
        "Give me information of the PetroVietnam?",
        # "What is the capial of Singapore?",
        # "What is the area of Block 16-1/15 from Neo4j Database",
    ]
    for q in questions:
        # Sequential Planner
        sequential_plan = await planner.create_plan(goal=q)
        print("The plan's steps are:")
        for step in sequential_plan._steps:
            print(
                f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
            )
        result = await sequential_plan.invoke(kernel)

        for res in result.metadata['results']:
            if str(type(res.value)) == "<class 'neo4j._work.eager_result.EagerResult'>":
                try:
                    print(res.value[0][0])
                except Exception as e:
                    print(e)
            if type(res.value) == list:
                print(res.value[0].inner_content.choices[0].message.content)

        return (result.metadata)

plugins = await main()


The plan's steps are:
- This is basic chat function, receiving normal conversation and returns the best fit user question, without domain knowledge requirements using ChatScripts-BasicChat with parameters: {'input': 'Give me information of the PetroVietnam'}
PetroVietnam, also known as Vietnam Oil and Gas Group, is a state-owned corporation in Vietnam. It is responsible for the exploration, production, and distribution of oil and gas resources in the country. PetroVietnam also engages in various other activities related to the petroleum industry, including refining, petrochemicals, and power generation. The company plays a significant role in the development of Vietnam's energy sector and contributes to the country's economic growth.


In [2]:
# Stepwise Function Calling
#--------------------------------------------
# options = FunctionCallingStepwisePlannerOptions(
#     max_iterations=10,
#     max_tokens=4000,
# )
# planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)
# for q in questions:
    # Stepwise Planner
    # result = await planner.invoke(kernel, q)
    # print(f"Q: {q}\nA: {result.final_answer}\n")

    # Uncomment the following line to view the planner's process for completing the request
    # print(f"\nChat history: {result.chat_history}\n")